In [ ]:
import pandas as pd
import chardet

In [ ]:
# initializing and cleaning individual datasets starting with smokers
smoker_data = pd.read_excel('Smoker_Data.xlsx')

#rename column to make it more logical when merged
smoker_data = smoker_data.rename(columns={"Proportion": "Proportion_smokers"})
smoker_data = smoker_data.rename(columns={"Current": "Current_smokers"})

smoker_data.head

#filter rest of columns out
smoker_data = smoker_data.loc[:, ["State", "Proportion_smokers", "Current_smokers"]]

In [ ]:
#now with veterans dataset

veterans_data = pd.read_excel('Veterans_by_State.xlsx')

veterans_data = veterans_data.rename(columns={"Grand Total": "number_of_veterans"})

veterans_data = veterans_data.loc[:, ["State", "number_of_veterans"]]

In [ ]:
#now with diabetes data set

diabetes_data = pd.read_csv('Diabetes_By_State.csv', skiprows=[0, 1])

diabetes_data = diabetes_data.rename(columns={"Percentage": "Percentage_diabetic"})

diabetes_data = diabetes_data.loc[:, ["State", "Percentage_diabetic"]]

In [ ]:
#merging above 3

merged_SmokDiabVet = smoker_data.merge(veterans_data, on='State').merge(diabetes_data, on = 'State')

In [ ]:
#condensing accidents data

accident_2018 = pd.read_csv('accident2018.csv')

accident_2018_condensed = accident_2018.groupby('STATENAME').size().reset_index(name='Accidents_2018')
accident_2018_condensed = accident_2018_condensed.rename(columns={"STATENAME": "State"})

In [ ]:
accident_2019 = pd.read_csv('accident2019.csv', encoding='ISO-8859-1')

accident_2019_condensed = accident_2019.groupby('STATENAME').size().reset_index(name='Accidents_2019')
accident_2019_condensed = accident_2019_condensed.rename(columns={"STATENAME": "State"})

accident_2019_condensed.head()

In [ ]:
#condensing accident 2020 data
accident_2020 = pd.read_csv('accident2020.csv', encoding='ISO-8859-1')

accident_2020_condensed = accident_2020.groupby('STATENAME').size().reset_index(name='Accidents_2020')
accident_2020_condensed = accident_2020_condensed.rename(columns={"STATENAME": "State"})

accident_2020_condensed.head()

In [ ]:
#merging accident data sets
merged_accidents = accident_2018_condensed.merge(accident_2019_condensed, on='State').merge(accident_2020_condensed, on = 'State')

merged_accidents.head()

In [ ]:
full_dataframe = merged_accidents.merge(merged_SmokDiabVet, on = 'State')

full_dataframe.head()

In [ ]:
#Data analysis

#this is the raw data size
tuples_list = [accident_2019.shape, accident_2020.shape, accident_2018.shape, smoker_data.shape, veterans_data.shape, diabetes_data.shape]
raw_data_size = tuple(sum(values) for values in zip(*tuples_list))
raw_data_size

In [ ]:
#save file
file_path = 'Data_Viz_FullData.csv'
full_dataframe.to_csv(file_path, index=False)

In [ ]:
#this is the cleaned data size
df = full_dataframe

df.shape

In [ ]:
# column names
df.columns

In [ ]:
# classificiation of each column
df.head()

In [ ]:
df.info()

In [ ]:
# fix datatype of percentage_diabetic
df['Percentage_diabetic'] = df['Percentage_diabetic'].astype('float64')
df.dtypes

In [ ]:
# quantitative data information
df.describe()

In [ ]:
# get info to easily display in writeup
def get_range(column):
  return column.max() - column.min()

for column in df.columns:
    # check if if the datatype is numeric! (documentation from online, need to exclude state)
    if pd.api.types.is_numeric_dtype(df[column]):
        # Calculate range, median, mean, and standard deviation
        col_range = get_range(df[column])
        col_median = df[column].median()
        col_mean = df[column].mean()
        col_std = df[column].std()

        # Print results
        print(f"Column: {column}")
        print(f"Range: {col_range}")
        print(f"Median: {col_median}")
        print(f"Mean: {col_mean}")
        print(f"Standard Deviation: {col_std}")
        print()

In [ ]:
# check for duplicates
df.head(100)

In [ ]:
# determine pearson correlation coefficients
df.corr()

In [ ]:
# going to factor for population, then run correlation
pops = pd.read_excel('nst-est2019-01.xlsx', header=0)
pops.head(100)

In [ ]:
# merge df and pops
df_pops = pd.merge(df, pops, on='State')
df_pops.head(10)

In [ ]:
# add rows to account for population
df_pops['Accidents_2018_proportion'] = df_pops['Accidents_2018'] / df_pops['Population']
df_pops['Accidents_2019_proportion'] = df_pops['Accidents_2019'] / df_pops['Population']
df_pops['Accidents_2020_proportion'] = df_pops['Accidents_2020'] / df_pops['Population']
df_pops['Current_smokers_proportion'] = df_pops['Current_smokers'] / df_pops['Population']
df_pops['number_of_veterans_proportion'] = df_pops['number_of_veterans'] / df_pops['Population']

In [ ]:
df_pops.head()

In [ ]:
# remove non-proportion columns
df_proportion = df_pops.drop(columns=['Accidents_2018', 'Accidents_2019', 'Accidents_2020', 'Current_smokers', 'number_of_veterans'])

In [ ]:
# now run pearson correlation coeff
df.corr()

In [ ]:
# import packages
import numpy as np
from scipy.optimize import curve_fit

In [ ]:
# Define linear, quadratic, and cubic functions
def linear(x, a, b):
    return a * x + b

def quadratic(x, a, b, c):
    return a * x**2 + b * x + c

def cubic(x, a, b, c, d):
    return a * x**3 + b * x**2 + c * x + d

# Fit the functions to the data
x_data = df['Accidents_2018'].values
y_data = df['Current_smokers'].values

linear_params, linear_covariance = curve_fit(linear, x_data, y_data)
quadratic_params, quadratic_covariance = curve_fit(quadratic, x_data, y_data)
cubic_params, cubic_covariance = curve_fit(cubic, x_data, y_data)

# Calculate the R-squared values
y_linear = linear(x_data, *linear_params)
y_quadratic = quadratic(x_data, *quadratic_params)
y_cubic = cubic(x_data, *cubic_params)

linear_r_squared = np.corrcoef(y_data, y_linear)[0, 1]**2
quadratic_r_squared = np.corrcoef(y_data, y_quadratic)[0, 1]**2
cubic_r_squared = np.corrcoef(y_data, y_cubic)[0, 1]**2

# Print the R-squared values
print(f"Accidents_2018 & current_smokers")
print(f"Linear R-squared: {linear_r_squared}")
print(f"Quadratic R-squared: {quadratic_r_squared}")
print(f"Cubic R-squared: {cubic_r_squared}")

In [ ]:
# Fit the functions to the data
x_data = df['Accidents_2018'].values
y_data = df['number_of_veterans'].values

linear_params, linear_covariance = curve_fit(linear, x_data, y_data)
quadratic_params, quadratic_covariance = curve_fit(quadratic, x_data, y_data)
cubic_params, cubic_covariance = curve_fit(cubic, x_data, y_data)

# Calculate the R-squared values
y_linear = linear(x_data, *linear_params)
y_quadratic = quadratic(x_data, *quadratic_params)
y_cubic = cubic(x_data, *cubic_params)

linear_r_squared = np.corrcoef(y_data, y_linear)[0, 1]**2
quadratic_r_squared = np.corrcoef(y_data, y_quadratic)[0, 1]**2
cubic_r_squared = np.corrcoef(y_data, y_cubic)[0, 1]**2

# Print the R-squared values
print(f"Accidents_2018 & number_of_veterans")
print(f"Linear R-squared: {linear_r_squared}")
print(f"Quadratic R-squared: {quadratic_r_squared}")
print(f"Cubic R-squared: {cubic_r_squared}")

In [ ]:
# Visuals
full_dataframe = df_pops

full_dataframe.head()

In [ ]:
#preparing data for analysis

full_dataframe['Proportion_diabetic'] = full_dataframe['Percentage_diabetic'] / 100

full_dataframe.head()

In [ ]:
proportion_columns = ['State', 'Accidents_2018_proportion', 'Accidents_2019_proportion', 'Accidents_2020_proportion',
                      'Current_smokers_proportion', 'number_of_veterans_proportion', 'Proportion_diabetic']

df_proportions = full_dataframe[proportion_columns]

# Display the new DataFrame
df_proportions.head()

In [ ]:
#employing min-max scaling to give all the proportions scores
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Select the columns to be scaled excluding 'State' 
columns_to_scale = [col for col in df_proportions.columns if col != 'State']

# Apply Min-Max scaling to the selected columns
scaled_data = scaler.fit_transform(df_proportions[columns_to_scale])

# Create a new df with the scaled data and the same column names excluding 'State'
df_scaled_values = pd.DataFrame(scaled_data, columns=columns_to_scale)

# Add the 'State' column back to the new df
df_scaled = pd.concat([df['State'], df_scaled_values], axis=1)

# Display
df_scaled.head()

In [ ]:
#Here I want to assign weights based on predictive capabilities: according to my research
#Car accidents account for 39% of amputations, Diabetic rates 25%, Smokers 12% and Veterans 7%, other unknown factors are the rest

# Create a list of weights 
# one weight for each column in df
weights = [0.13, 0.13, 0.13, 0.12, 0.07, 0.25]

weights_dict = {col: weight for col, weight in zip(df_scaled.columns[1:], weights)}

# Multiply each column except 'State' in the df by its corresponding weight
df_weighted = df_scaled.copy()
for col, weight in weights_dict.items():
    df_weighted[col] = df_scaled[col] * weight

# Display the weighted df
df_weighted.head()

In [ ]:
# create a total risk score for amputations and demand score for prosthetics
# Compute the total score for each row by summing the values across columns excluding the 'State' column
df_weighted['total_score'] = df_weighted.drop('State', axis=1).sum(axis=1)

# Display the updated df with the 'total_score' column
df_weighted.head()

In [ ]:
#Scale the risk score
scaler = MinMaxScaler(feature_range=(0, 10))

# Reshape the 'total_score' column to a 2D array and apply Min-Max scaling
scaled_total_score = scaler.fit_transform(df_weighted[['total_score']])

# Replace the 'total_score' column with the scaled data
df_weighted['total_score'] = scaled_total_score

# Display the updated df with the scaled 'total_score' column
df_weighted.head()

In [ ]:
# plotly map
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
!pip install plotly-orca
!pip install "notebook>=5.3" "ipywidgets>=7.2"

In [ ]:
!pip install kaleido

In [ ]:
import plotly.io as pio

In [ ]:
# Dictionary to map full state names to abbreviations
state_abbr = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Convert full state names to abbreviations
df_weighted['State'] = df_weighted['State'].map(state_abbr)

# Create a choropleth map using Plotly Express
fig = px.choropleth(df_weighted, 
                    locations='State', 
                    locationmode='USA-states', 
                    color='total_score', 
                    scope='usa', 
                    color_continuous_scale='Viridis', 
                    labels={'total_score': 'Demand Score'},
                    title='Demand by State')

fig.show()

In [ ]:
#fig.write_html("visual_two.html")
#pio.write_image(fig, "visual_two.png", format='png', engine='kaleido')

In [ ]:
# List of columns to plot excluding 'State' and 'total_score'
columns_to_plot = [col for col in df_weighted.columns if col not in ['State', 'total_score']]

# Create a Choropleth trace for each column
traces = [go.Choropleth(
    locations=df_weighted['State'],
    z=df_weighted[col],
    locationmode='USA-states',
    colorscale='Viridis',
    visible=False,
    name=col,
    showscale=True, 
    hovertemplate=f"{col}: %{col}<extra></extra>"
) for col in columns_to_plot]

# Create a Figure with the Choropleth traces
fig = go.Figure(data=traces)

# Set the layout to display the map of the USA and add a title
fig.update_layout(
    title='Choropleth Map of Columns',
    geo=dict(scope='usa', projection=dict(type='albers usa')),
    margin=dict(l=0, r=0, b=0, t=30)
)

# Add a checkbox updatemenu to control the visibility of each trace
fig.update_layout(
    updatemenus=[
        dict(
            type='buttons',
            showactive=True,
            buttons=[dict(label=col,
                          method='update',
                          args=[{'visible': [col == trace.name for trace in traces], 
                                 'showscale': [True for _ in traces]  
                                 }])
                     for col in columns_to_plot]
        )
    ]
)

fig.show()

In [ ]:
df_pops = df_pops.reset_index()

In [ ]:
# get data by year
df_pops.head()
accidents = df_pops[["State", "Accidents_2018_proportion", "Accidents_2019_proportion", "Accidents_2020_proportion"]]
accidents.head()

# Select the columns to be scaled excluding the 'State' column
columns_to_scale = [col for col in accidents.columns if col != 'State']

# Apply Min-Max scaling to the selected columns
scaled_data = scaler.fit_transform(accidents[columns_to_scale])

# Create a new df with the scaled data and the same column names excluding 'State'
df_scaled_values = pd.DataFrame(scaled_data, columns=columns_to_scale)

# Add the 'State' column back to the new DataFrame
df_scaled = pd.concat([df['State'], df_scaled_values], axis=1)

# Display the scaled df
df_scaled.head()

accidents_scaled = df_scaled
accidents_scaled

newcolnames = {'Accidents_2018_proportion': '2018', 'Accidents_2019_proportion': '2019', 'Accidents_2020_proportion': '2020'}
accidents_scaled = accidents_scaled.rename(columns=newcolnames)
accidents_scaled.head()

accidents_scaled["change"] = accidents_scaled["2020"] - accidents_scaled["2018"]
acc = accidents_scaled
acc.head()

In [ ]:
import altair as alt
from vega_datasets import data

# Load US States GeoJSON data
us_states = alt.topo_feature(data.us_10m.url, 'states')

In [ ]:
# Dictionary to map state abbreviations to FIPS codes
state_name_to_fips = {
    'Alabama': '1',
    'Alaska': '2',
    'Arizona': '4',
    'Arkansas': '5',
    'California': '6',
    'Colorado': '8',
    'Connecticut': '9',
    'Delaware': '10',
    'Florida': '12',
    'Georgia': '13',
    'Hawaii': '15',
    'Idaho': '16',
    'Illinois': '17',
    'Indiana': '18',
    'Iowa': '19',
    'Kansas': '20',
    'Kentucky': '21',
    'Louisiana': '22',
    'Maine': '23',
    'Maryland': '24',
    'Massachusetts': '25',
    'Michigan': '26',
    'Minnesota': '27',
    'Mississippi': '28',
    'Missouri': '29',
    'Montana': '30',
    'Nebraska': '31',
    'Nevada': '32',
    'New Hampshire': '33',
    'New Jersey': '34',
    'New Mexico': '35',
    'New York': '36',
    'North Carolina': '37',
    'North Dakota': '38',
    'Ohio': '39',
    'Oklahoma': '40',
    'Oregon': '41',
    'Pennsylvania': '42',
    'Rhode Island': '44',
    'South Carolina': '45',
    'South Dakota': '46',
    'Tennessee': '47',
    'Texas': '48',
    'Utah': '49',
    'Vermont': '50',
    'Virginia': '51',
    'Washington': '53',
    'West Virginia': '54',
    'Wisconsin': '55',
    'Wyoming': '56'
}

acc['id'] = acc['State'].map(state_name_to_fips)
acc['Accidents'] = acc['change']

state_data = acc

In [ ]:
# Define the color scale for the choropleth map
color_scale = alt.Scale(scheme='viridis', domain=[state_data['Accidents'].min(), state_data['Accidents'].max()])

# Create the choropleth map
choropleth_map = alt.Chart(us_states).mark_geoshape().encode(
    alt.Color('Accidents:Q', scale=color_scale, legend=alt.Legend(title='Accidents')),
    tooltip=['Accidents:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(state_data, 'id', ['Accidents'])
).project(
    type='albersUsa'
).properties(
    width=700,
    height=400,
    title='Increase in Accidents from 2018 to 2020'
)

choropleth_map.display()

In [ ]:
choropleth_map.save("visual_three.html")

# Install altair_saver
#!pip install altair_saver

# Import altair_saver
#import altair_saver

# Save the chart as a PNG
#altair_saver.save(choropleth_map, "visual_three.png")

In [ ]:
# finding supply side data

# Read the Excel file
file_path = 'ProsthOrth_Data.xlsx'
supply_df = pd.read_excel(file_path, skiprows=5, nrows=48)

supply_df.head()

In [ ]:
# wrangling

supply_df = supply_df.rename(columns={'Area Name': 'State'})

supply_df.head()

In [ ]:
# clean the data of any state values that are blank or list them as 0(?)

# List of states to drop
states_to_drop = ['Alaska', 'Colorado', 'Delaware', 'Vermont', 'Wisconsin', 'District of Columbia']

# Remove rows containing the specified states
cleaned_supply_df = supply_df[~supply_df['State'].isin(states_to_drop)]

# Reset the index after removing the rows
cleaned_supply_df.reset_index(drop=True, inplace=True)

# Display the cleaned dataframe
# print(cleaned_supply_df)

# rename columns

# Rename the 'Employment(1)' column
cleaned_supply_df = cleaned_supply_df.rename(columns={'Employment(1)': 'Number of Prosthetists'})

# Display the updated dataframe
cleaned_supply_df.head()

In [ ]:
# scale these values by population

cleaned_supply_df = cleaned_supply_df.merge(pops, on='State')

In [ ]:
cleaned_supply_df.head()

In [ ]:
#now add a column to find the prosthetists by capita

cleaned_supply_df['Number of Prosthetists'] = pd.to_numeric(cleaned_supply_df['Number of Prosthetists'], errors='coerce')
cleaned_supply_df['Population'] = pd.to_numeric(cleaned_supply_df['Population'], errors='coerce')

# Calculate the 'prosthetists per capita' column
cleaned_supply_df['prosthetists per capita'] = cleaned_supply_df['Number of Prosthetists'] / cleaned_supply_df['Population']

# Display the updated df
cleaned_supply_df.head()

In [ ]:
#min max scale

scaler = MinMaxScaler(feature_range=(0, 10))

# Scale the 'prosthetists per capita' column using the scaler and create the 'Supply Score' column with the scaled values
cleaned_supply_df['Supply Score'] = scaler.fit_transform(cleaned_supply_df[['prosthetists per capita']])

In [ ]:
cleaned_supply_df.head()

In [ ]:
!pip install geopandas matplotlib

In [ ]:
# Convert full state names to abbreviations
cleaned_supply_df['State'] = cleaned_supply_df['State'].map(state_abbr)

# Create a choropleth map using Plotly Express
fig = px.choropleth(cleaned_supply_df, 
                    locations='State', 
                    locationmode='USA-states', 
                    color='Supply Score', 
                    scope='usa', 
                    color_continuous_scale='Viridis', 
                    labels={'Supply Score': 'Supply Score'},
                    title='Supply Scores by State')

# Show the plot
fig.show()
# 10 = lots of prostethists

In [ ]:
# fig.write_html("visual_four.html")
# pio.write_image(fig, "visual_four.png", format='png', engine='kaleido')

In [ ]:
cleaned_supply_df.head()

In [ ]:
df_weighted.head()

In [ ]:
# create a df that has supply and demand scores so we can compare/contrast them

# Merge the df on the "State" column
SupplyVSDemand_df = pd.merge(df_weighted, cleaned_supply_df, on='State')

# Display the new df
SupplyVSDemand_df.head()

In [ ]:
#make new column for opportunity zones
SupplyVSDemand_df['Opportunity Score'] = SupplyVSDemand_df['total_score'] - SupplyVSDemand_df['Supply Score']

# Display the updated df
SupplyVSDemand_df.head()

In [ ]:
# Create a choropleth map using Plotly Express
fig = px.choropleth(SupplyVSDemand_df, 
                    locations='State', 
                    locationmode='USA-states', 
                    color='Opportunity Score', 
                    scope='usa', 
                    color_continuous_scale='Viridis', 
                    labels={'Opportunity Score': 'Opportunity Score'},
                    title='Opportunity Scores by State')

# Show the plot
fig.show()

In [ ]:
# fig.write_html("visual_six.html")
# pio.write_image(fig, "visual_six.png", format='png', engine='kaleido')

In [ ]:
# Make a chart of accidents
df.head()
df_accidents = df[['State','Accidents_2018', 'Accidents_2019', 'Accidents_2020']]
df_accidents.head()
df_accidents = df_accidents.rename(columns={'Accidents_2018':2018, 'Accidents_2019':2019, 'Accidents_2020':2020})
df_accidents.head()

In [ ]:
df_melted = df_accidents.melt(id_vars=['State'], value_vars=[2018, 2019, 2020], var_name='Year', value_name='Value')

df_sorted = df_melted.sort_values("State")
df_sorted = df_sorted.rename(columns={"Value":"Accidents"})
df_sorted

In [ ]:
# create a slider input
slider = alt.binding_range(min=2018, max=2020, step=1)

slider_selection = alt.selection_single(bind=slider, fields=['Year'], name="Select", value=2018)

# create a chart
car_accidents = alt.Chart(df_sorted).mark_bar().encode(
    y=alt.Y('State:N', sort='-x'),
    x='Accidents:Q',
    color=alt.Color('State:N', legend=None)
).properties(
    width=500,
    height=600,
    title='Car Accidents by State (2018-2020)'
).add_selection(
    slider_selection
).transform_filter(
    slider_selection
)

car_accidents

In [ ]:
# Read in the CSV file
df_income = pd.read_csv('incomedata.csv')

# Display the first few rows of the dataset
df_income.head()

In [ ]:
import plotly.io as pio
import plotly.express as px

In [ ]:
df_income = pd.read_csv('incomedata.csv')

# Remove the row with "state" as "District of Columbia"
df_income = df_income[df_income['state'] != 'District of Columbia']

# Convert full state names to abbreviations
df_income['state'] = df_income['state'].map(state_abbr)

# Create a color map with the income distribution of each state
fig = px.choropleth(
    df_income,
    locations='state',
    locationmode='USA-states',
    color='HouseholdIncome',
    scope='usa',
    title='Median Household Income by State',
    hover_name='state',
    color_continuous_scale= 'Viridis',
    labels={'HouseholdIncome': 'Median Household Income'}
)

# Show the plot
fig.show()

In [ ]:
pip install pandas openpyxl

In [ ]:
# Read the Excel file into a pandas df
df = pd.read_excel('MKTPROSDATA.xlsx', engine='openpyxl')

# Print the DataFrame to see the data
df.head()

In [ ]:
import plotly.graph_objs as go

In [ ]:
# Melt the df to make it suitable for plotting
df_melted = df.melt(id_vars='Region', var_name='Year', value_name='Prosthetics Sold')

# Create traces for each region
traces = []
for region in df_melted['Region'].unique():
    trace = go.Scatter(
        x=df_melted[df_melted['Region'] == region]['Year'],
        y=df_melted[df_melted['Region'] == region]['Prosthetics Sold'],
        mode='lines+markers',
        name=region,
        visible=(region == 'U.S') 
    )
    traces.append(trace)

# Create a layout with a dropdown menu
layout = go.Layout(
    title='Projected Prosthetics Sold by Region (2021-2040)',
    xaxis={'title': 'Year'},
    yaxis={'title': 'Number of Prosthetics Sold'},
    updatemenus=[
        {
            'buttons': [
                {
                    'label': 'All Regions',
                    'method': 'update',
                    'args': [
                        {'visible': [True, True, True]},
                        {'title': 'Projected Prosthetics Sold by Region (2021-2040)'}
                    ]
                }
            ] + [
                {
                    'label': region,
                    'method': 'update',
                    'args': [
                        {'visible': [region == r for r in df_melted['Region'].unique()]},
                        {'title': f'Projected Prosthetics Sold in {region} (2021-2040)'}
                    ]
                }
                for region in df_melted['Region'].unique()
            ],
            'direction': 'down',
            'showactive': True
        }
    ]
)

# Create a Figure and show the plot
fig = go.Figure(data=traces, layout=layout)
fig.show()

In [ ]:
df2 = pd.read_excel('MKTLIMBLDATA.xlsx', engine='openpyxl')

# Print the DataFrame to see the data
df2.head()

In [ ]:
# Melt the DataFrame to make it suitable for plotting
df_melted2 = df2.melt(id_vars='Region', var_name='Year', value_name='Limb Loss')

# Create traces for each region
traces = []
for region in df_melted['Region'].unique():
    trace = go.Scatter(
        x=df_melted2[df_melted2['Region'] == region]['Year'],
        y=df_melted2[df_melted2['Region'] == region]['Limb Loss'],
        mode='lines+markers',
        name=region,
        visible=(region == 'U.S')  # Show only the 'U.S' trace initially
    )
    traces.append(trace)

# Create a layout with a dropdown menu
layout = go.Layout(
    title='Projected Limb Loss by Region (2021-2040)',
    xaxis={'title': 'Year'},
    yaxis={'title': 'Limb Loss'},
    updatemenus=[
        {
            'buttons': [
                {
                    'label': 'All Regions',
                    'method': 'update',
                    'args': [
                        {'visible': [True, True, True]},
                        {'title': 'Projected Limb Loss by Region (2021-2040)'}
                    ]
                }
            ] + [
                {
                    'label': region,
                    'method': 'update',
                    'args': [
                        {'visible': [region == r for r in df_melted2['Region'].unique()]},
                        {'title': f'Projected Limb Loss in {region} (2021-2040)'}
                    ]
                }
                for region in df_melted2['Region'].unique()
            ],
            'direction': 'down',
            'showactive': True
        }
    ]
)

# Create a Figure and show the plot
fig = go.Figure(data=traces, layout=layout)
fig.show()

In [ ]:
df_melted = df.melt(id_vars='Region', var_name='Year', value_name='Prosthetics Sold')
df2_melted = df2.melt(id_vars='Region', var_name='Year', value_name='Limb Loss')

# Create traces for prosthetics sold
prosthetics_traces = []
for region in df_melted['Region'].unique():
    trace = go.Scatter(
        x=df_melted[df_melted['Region'] == region]['Year'],
        y=df_melted[df_melted['Region'] == region]['Prosthetics Sold'],
        mode='lines+markers',
        name=f'Prosthetics Sold ({region})',
        marker=dict(symbol='circle'),
        visible=(region == 'U.S')  # Show only the 'U.S' trace initially
    )
    prosthetics_traces.append(trace)

# Create traces for limb loss
limb_loss_traces = []
for region in df2_melted['Region'].unique():
    trace = go.Scatter(
        x=df2_melted[df2_melted['Region'] == region]['Year'],
        y=df2_melted[df2_melted['Region'] == region]['Limb Loss'],
        mode='lines+markers',
        name=f'Limb Loss ({region})',
        marker=dict(symbol='square'),
        line=dict(dash='dash'),
        visible=False
    )
    limb_loss_traces.append(trace)

# Combine the traces
all_traces = prosthetics_traces + limb_loss_traces

# Create a layout with a dropdown menu
layout = go.Layout(
    title='Projected Prosthetics Sold and Limb Loss by Region (2021-2040)',
    xaxis={'title': 'Year'},
    yaxis={'title': 'Number of Sold/Needed'},
    updatemenus=[
        {
            'buttons': [
                {
                    'label': 'All Regions',
                    'method': 'update',
                    'args': [
                        {'visible': [True, True, True] * 2},
                        {'title': 'Projected Prosthetics Sold and Limb Loss by Region (2021-2040)'}
                    ]
                }
            ] + [
                {
                    'label': region,
                    'method': 'update',
                    'args': [
                        {'visible': [(region == r) or (region == r2) for r, r2 in zip(df_melted['Region'].unique(), df2_melted['Region'].unique())] * 2},
                        {'title': f'Projected Prosthetics Sold and Limb Loss in {region} (2021-2040)'}
                    ]
                }
                for region in df_melted['Region'].unique()
            ],
            'direction': 'down',
            'showactive': True
        }
    ]
)

# Create a Figure and show the plot
fig = go.Figure(data=all_traces, layout=layout)
fig.show()